In [32]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [2]:
coffee = pd.read_csv('../data/coffee_fix_clean.csv')

In [3]:
coffee = coffee.loc[:,['name','roaster','roaster_city','roaster_country_state','roaster_country','origin',
                       'origin_count','origin_country','review_date','est_price','rating','roast','agtron',
                       'agtron_grounds','agtron_bean','aroma','acid','body','flavor','aftertaste','with_milk',
                       'desc_1','desc_2','desc_3']]

In [4]:
origin_feature=pd.get_dummies(coffee['origin_country'])

In [34]:
origin_feature

,Africa,Africa; Asia,Africa; Asia Pacific,Africa; Central America,Africa; Indonesia; South America,Africa; South America,Africa; South America; Central America,America; Indo-Pacific,Asia; Africa; America,Bolivia,...,South America,Taiwan,Tanzania,Thailand,Thailand; Laos,Thailand; Vietnam; Latin America,Uganda,Vietnam,Yemen,Zambia
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2273,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2274,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2275,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
neighbour_origin = NearestNeighbors()

In [7]:
user_input = 'Colombia'

In [29]:
neighbour_origin.fit(origin_feature.loc[:,[user_input]])

NearestNeighbors()

In [33]:
index = neighbour_origin.kneighbors([[1]], 2, return_distance=False)

In [25]:
select_index = np.reshape(index,-1)

In [26]:
coffee.iloc[select_index,:]

,name,roaster,roaster_city,roaster_country_state,roaster_country,origin,origin_count,origin_country,review_date,est_price,...,agtron_bean,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
42,Static Colombia Decaf Midnight,Espresso Republic,Chino,California,United States,"Huila, Colombia",1,Colombia,December 2017,$16.95/12 ounces,...,58,8.0,8.0,8.0,8.0,7.0,0.0,"Deeply sweet, chocolate-toned. Dark chocolate,...",Decaffeinated at the Descamex plant in Mexico ...,"A solid, chocolate-toned decaf with inviting f..."
22,Decaf Colombia Select,Bootstrap Coffee Roasters,St. Paul,Minnesota,United States,Colombia,1,Colombia,November 2017,$15.00/12 ounces,...,76,7.0,7.0,8.0,8.0,7.0,0.0,"Sweet, delicate, floral and wood-toned. Fresh-...",Produced from trees of the Caturra and Castill...,Those who value a gently zesty sweetness shoul...


In [12]:
taste_feature = coffee.loc[:,['aroma', 'acid', 'body', 'flavor', 'aftertaste']]

In [36]:
taste_feature

,aroma,acid,body,flavor,aftertaste
0,9.0,0.0,9.0,9.0,9.0
1,9.0,0.0,9.0,9.0,8.0
2,9.0,8.0,8.0,9.0,8.0
3,8.0,8.0,9.0,9.0,8.0
4,9.0,9.0,9.0,9.0,8.0
...,...,...,...,...,...
2272,8.0,8.0,9.0,9.0,8.0
2273,9.0,9.0,8.0,9.0,8.0
2274,9.0,9.0,8.0,9.0,8.0
2275,9.0,8.0,8.0,9.0,8.0


In [13]:
neighbour_taste = NearestNeighbors()

In [14]:
neighbour_taste.fit(taste_feature)

NearestNeighbors()

In [19]:
taste_input = [[5,2,8,5,9]]

In [35]:
taste_index = neighbour_taste.kneighbors(taste_input, 2, return_distance=False)

In [21]:
select_taste_index = np.reshape(taste_index,-1)

In [22]:
coffee.iloc[select_taste_index,:]

,name,roaster,roaster_city,roaster_country_state,roaster_country,origin,origin_count,origin_country,review_date,est_price,...,agtron_bean,aroma,acid,body,flavor,aftertaste,with_milk,desc_1,desc_2,desc_3
758,Blend Espresso,"Nongfu Spring Co., Ltd.",Hangzhou,Hangzhou,China,Brazil; Ethiopia,2,Brazil; Ethiopia,December 2019,$12.00/500 grams,...,55,7.0,0.0,7.0,7.0,7.0,7.0,"Evaluated as espresso. Sweetly tart, smoky. Ca...",An espresso blend comprised of coffees from Br...,"An accessible, bittersweet espresso blend with..."
16,“Nutty” Espresso Blend,A.R.C.,Hong Kong,Hong Kong,Hong Kong,Not disclosed,1,Not disclosed,November 2017,HKD $130/454 grams,...,68,7.0,0.0,8.0,8.0,7.0,8.0,"Evaluated as espresso. Fir, almond, baker’s ch...","A.R.C., whose motto is “more than specialty,” ...","Nut and chocolate dominate in this crisp, bris..."
